In [1]:
import gibbs
import numpy as np

In [2]:
N = 3  # neurons
K = 2  # states
T = 50000  # time steps
seed = 1234 
num_rep = 10


pi=np.array([1,0])

gamma=np.array([[0.9,0.1],
                [0.1,0.9]])

r=np.array([[1,0],
            [0,1],
            [1,0],
            [0,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K (K=2)"


gibbs.run_simulation(T, N, K, seed, num_rep, pi, gamma, r, type_run)

Data is generated according to an HMM with the following parameters:
Pi:
 [1 0]
Gamma:
 [[0.9 0.1]
 [0.1 0.9]]
R:
 [[0.5 0. ]
 [0.  0.5]
 [0.5 0. ]
 [0.  0.5]]
50000 data points are generated, which are:
Generated firings: [0 2 0 ... 3 1 1]
Generated states: [0 0 0 ... 1 1 1]
Rep 0...
##########################
Estimated values for the parameters
Estimated Pi =  [0.94788856 0.05211144]
Estimated Gamma =  [[0.90074909 0.09925091]
 [0.09460136 0.90539864]]
Estimated R =  [[0.5008321  0.01599876]
 [0.00871608 0.47252661]
 [0.4827445  0.03088506]
 [0.00770732 0.48058957]]
Rep 1...
##########################
Estimated values for the parameters
Estimated Pi =  [0.45869715 0.54130285]
Estimated Gamma =  [[0.75982564 0.24017436]
 [0.18473256 0.81526744]]
Estimated R =  [[0.19696876 0.29616715]
 [0.44783446 0.0905674 ]
 [0.06904231 0.39243481]
 [0.28615448 0.22083063]]
Rep 2...
##########################
Estimated values for the parameters
Estimated Pi =  [0.49580257 0.50419743]
Estimated Gamma

In [ ]:
N = 3  # neurons
K = 4  # states
T = 50000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0])
gamma=np.array([[0.85,0.05,0.05,0.05],
                [0.05,0.85,0.05,0.05],
                [0.05,0.05,0.85,0.05],
                [0.05,0.05,0.05,0.85]])
r=np.array([[1,0,1,0],
            [0,1,1,0],
            [1,0,0,1],
            [0,1,0,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K (K=4)"


gibbs.run_simulation(T, N, K, seed, num_rep, pi, gamma, r, type_run)

In [ ]:
N = 3  # neurons
K = 5  # states
T = 50000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0,0])
gamma=np.array([[0.8,0.05,0.05,0.05,0.05],
                [0.05,0.8,0.05,0.05,0.05],
                [0.05,0.05,0.8,0.05,0.05],
                [0.05,0.05,0.05,0.8,0.05],
                [0.05,0.05,0.05,0.05,0.8]])
r=np.array([[1,0,1,0,1],
            [0,1,1,0,0],
            [1,0,0,1,1],
            [0,1,0,1,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K (K=5)"


gibbs.run_simulation(T, N, K, seed, num_rep, pi, gamma, r, type_run)


In [ ]:
N = 3  # neurons
K = 6  # states
T = 50000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0,0,0])
gamma=np.array([[0.75,0.05,0.05,0.05,0.05,0.05],
                [0.05,0.75,0.05,0.05,0.05,0.05],
                [0.05,0.05,0.75,0.05,0.05,0.05],
                [0.05,0.05,0.05,0.75,0.05,0.05],
                [0.05,0.05,0.05,0.05,0.75,0.05],
                [0.05,0.05,0.05,0.05,0.05,0.75]])
r=np.array([[1,0,1,0,1,1],
            [0,1,1,0,0,1],
            [1,0,0,1,1,0],
            [0,1,0,1,1,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K (K=6)"

gibbs.run_simulation(T, N, K, seed, num_rep, pi, gamma, r, type_run)

In [ ]:
N = 3  # neurons
K = 7  # states
T = 50000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0,0,0,0])
gamma=np.array([[0.7,0.05,0.05,0.05,0.05,0.05,0.05],
                [0.05,0.7,0.05,0.05,0.05,0.05,0.05],
                [0.05,0.05,0.7,0.05,0.05,0.05,0.05],
                [0.05,0.05,0.05,0.7,0.05,0.05,0.05],
                [0.05,0.05,0.05,0.05,0.7,0.05,0.05],
                [0.05,0.05,0.05,0.05,0.05,0.7,0.05],
                [0.05,0.05,0.05,0.05,0.05,0.05,0.7]])
r=np.array([[1,0,1,0,1,1,1],
            [0,1,1,0,0,1,1],
            [1,0,0,1,1,0,1],
            [0,1,0,1,1,1,0]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K (K=7)"

gibbs.run_simulation(T, N, K, seed, num_rep, pi, gamma, r, type_run)

In [ ]:
N = 3  # neurons
K = 8  # states
T = 50000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0,0,0,0,0])
gamma=np.array([[0.72,0.04,0.04,0.04,0.04,0.04,0.04,0.04],
                [0.04,0.72,0.04,0.04,0.04,0.04,0.04,0.04],
                [0.04,0.04,0.72,0.04,0.04,0.04,0.04,0.04],
                [0.04,0.04,0.04,0.72,0.04,0.04,0.04,0.04],
                [0.04,0.04,0.04,0.04,0.72,0.04,0.04,0.04],
                [0.04,0.04,0.04,0.04,0.04,0.72,0.04,0.04],
                [0.04,0.04,0.04,0.04,0.04,0.04,0.72,0.04],
                [0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.72]])
r=np.array([[1,0,1,0,1,1,1,0],
            [0,1,1,0,0,1,1,1],
            [1,0,0,1,1,0,1,1],
            [0,1,0,1,1,1,0,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K (K=8)"

gibbs.run_simulation(T, N, K, seed, num_rep, pi, gamma, r, type_run)

## RUN CON N=K+1 ##

In [ ]:
N = 4  # neurons
K = 3  # states
T = 50000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0])
gamma=np.array([[0.9,0.05,0.05],
                [0.05,0.9,0.05],
                [0.05,0.05,0.9]])
r=np.array([[1,0,0],
            [0,1,0],
            [1,0,1],
            [0,1,0],
            [0,0,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K and N (K=3, N=4)"


gibbs.run_simulation(T,N,K,seed,num_rep,pi,gamma,r,type_run)

In [ ]:
N = 5  # neurons
K = 4  # states
T = 100000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0])
gamma=np.array([[0.85,0.05,0.05,0.05],
                [0.05,0.85,0.05,0.05],
                [0.05,0.05,0.85,0.05],
                [0.05,0.05,0.05,0.85]])
r=np.array([[1,0,0,0],
            [0,1,0,0],
            [1,0,1,0],
            [0,1,0,1],
            [0,0,1,0],
            [0,0,0,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K and N (K=4, N=5)"


gibbs.run_simulation(T,N,K,seed,num_rep,pi,gamma,r,type_run)

In [ ]:
N = 6  # neurons
K = 5  # states
T = 100000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0,0])
gamma=np.array([[0.8,0.05,0.05,0.05,0.05],
                [0.05,0.8,0.05,0.05,0.05],
                [0.05,0.05,0.8,0.05,0.05],
                [0.05,0.05,0.05,0.8,0.05],
                [0.05,0.05,0.05,0.05,0.8]])
r=np.array([[1,0,0,0,0],
            [0,1,0,0,0],
            [1,0,1,0,0],
            [0,1,0,1,0],
            [0,0,1,0,1],
            [0,0,0,1,0],
            [0,0,0,0,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K and N (K=5, N=6)"


gibbs.run_simulation(T,N,K,seed,num_rep,pi,gamma,r,type_run)

In [ ]:
N = 7  # neurons
K = 6  # states
T = 100000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0,0,0])
gamma=np.array([[0.75,0.05,0.05,0.05,0.05,0.05],
                [0.05,0.75,0.05,0.05,0.05,0.05],
                [0.05,0.05,0.75,0.05,0.05,0.05],
                [0.05,0.05,0.05,0.75,0.05,0.05],
                [0.05,0.05,0.05,0.05,0.75,0.05],
                [0.05,0.05,0.05,0.05,0.05,0.75]])
r=np.array([[1,0,0,0,0,0],
            [0,1,0,0,0,0],
            [1,0,1,0,0,0],
            [0,1,0,1,0,0],
            [0,0,1,0,1,0],
            [0,0,0,1,0,1],
            [0,0,0,0,1,0],
            [0,0,0,0,0,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K and N (K=6, N=7)"


gibbs.run_simulation(T,N,K,seed,num_rep,pi,gamma,r,type_run)

In [ ]:
N = 8  # neurons
K = 7  # states
T = 100000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0,0,0,0])
gamma=np.array([[0.7,0.05,0.05,0.05,0.05,0.05,0.05],
                [0.05,0.7,0.05,0.05,0.05,0.05,0.05],
                [0.05,0.05,0.7,0.05,0.05,0.05,0.05],
                [0.05,0.05,0.05,0.7,0.05,0.05,0.05],
                [0.05,0.05,0.05,0.05,0.7,0.05,0.05],
                [0.05,0.05,0.05,0.05,0.05,0.7,0.05],
                [0.05,0.05,0.05,0.05,0.05,0.05,0.7]])
r=np.array([[1,0,0,0,0,0,0],
            [0,1,0,0,0,0,0],
            [1,0,1,0,0,0,0],
            [0,1,0,1,0,0,0],
            [0,0,1,0,1,0,0],
            [0,0,0,1,0,1,0],
            [0,0,0,0,1,0,1],
            [0,0,0,0,0,1,0],
            [0,0,0,0,0,0,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K and N (K=7, N=8)"


gibbs.run_simulation(T,N,K,seed,num_rep,pi,gamma,r,type_run)

In [ ]:
N = 9  # neurons
K = 8  # states
T = 100000  # time steps
seed = None #1234 
num_rep = 10

#input: initialization of the parameters for the HMM
#pi, gamma, r = initialize_parameters(N, K, seed)

pi=np.array([1,0,0,0,0,0,0,0])
gamma=np.array([[0.72,0.04,0.04,0.04,0.04,0.04,0.04,0.04],
                [0.04,0.72,0.04,0.04,0.04,0.04,0.04,0.04],
                [0.04,0.04,0.72,0.04,0.04,0.04,0.04,0.04],
                [0.04,0.04,0.04,0.72,0.04,0.04,0.04,0.04],
                [0.04,0.04,0.04,0.04,0.72,0.04,0.04,0.04],
                [0.04,0.04,0.04,0.04,0.04,0.72,0.04,0.04],
                [0.04,0.04,0.04,0.04,0.04,0.04,0.72,0.04],
                [0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.72]])
r=np.array([[1,0,0,0,0,0,0,0],
            [0,1,0,0,0,0,0,0],
            [1,0,1,0,0,0,0,0],
            [0,1,0,1,0,0,0,0],
            [0,0,1,0,1,0,0,0],
            [0,0,0,1,0,1,0,0],
            [0,0,0,0,1,0,1,0],
            [0,0,0,0,0,1,0,1],
            [0,0,0,0,0,0,1,0],
            [0,0,0,0,0,0,0,1]])


type_run = "Complexity increased keeping gamma very unbalanced, and increasing K and N (K=8, N=9)"


gibbs.run_simulation(T,N,K,seed,num_rep,pi,gamma,r,type_run)